PROCESO ML

#### Bibliotecas

In [1]:
import bootcampviztools as bt
import hashlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modulo_toolbox as mtb
import requests
import seaborn as sns
import time
import toolbox_ML as tb
import warnings

warnings.filterwarnings('ignore')

# API MARVEL

In [2]:
# Claves de la API
public_key = '0399714820d7f8388e74266a3c94cb4a'
private_key = '7fe33c6de400b6bafdd3ba7f8b3158681c092aed'

In [4]:
# Parametros de autenticacion
def get_hash_params(public_key, private_key):
    ts = str(time.time()) 
    hash_string = ts + private_key+ public_key 
    hash_result = hashlib.md5(hash_string.encode()).hexdigest()
    return {
        'ts':ts,
        'apikey': public_key,
        'hash': hash_result,
    }

In [5]:
# Funcion obtener todos los personajes
def fetch_all_characters():
        base_url = 'http://gateway.marvel.com/v1/public/characters'
        all_characters = []
        limit = 100
        offset = 0

        while True:
                params = get_hash_params(public_key, private_key)
                params.update({
                        'limit': limit,
                        'offset': offset
                })

                response = requests.get(base_url, params = params)

                if response.status_code != 200:
                        print(f'Error: {response.status_code}')
                        break
                
                data = response.json()
                characters = data['data']['results']

                if not characters:
                        break

                all_characters.extend(characters)
                ofset += limit

                # Mostrar progreso
                print(f"Obtenidos {len(all_characters)} personajes...")
                
                # Pausa para respetar los limite de la API
                time.sleep(1)
        
        return all_characters

In [6]:
# Funcion convertir datos en un dataframe
def characters_to_dataframe(characters):
    character_data = []
    for character in characters:
        character_data.append({
            'id': character['id'],
            'name': character['name'],
            'description': character['description'],
            'modified': character['modified'], 
            'comics_available': character['comics']['available'],
            'series_available': character['series']['available'],
            'stories_available': character['stories']['available'],
            'events_available': character['stories']['available'],
            'event_available': character['events']['available'], 
            'thumbnail': f"{character['thumbnail']['path']}.{character['thumbnail']['extension']}"
        })
    df = pd.DataFrame(character_data)
    return df

In [14]:
# Obtener personajes
all_characters = fetch_all_characters()

# Covnertir datos en un Dataframe
characters_df = characters_to_dataframe(all_characters)
characters_df.head()

Obtenidos 100 personajes...
Obtenidos 200 personajes...
Obtenidos 300 personajes...
Obtenidos 400 personajes...
Obtenidos 500 personajes...
Obtenidos 600 personajes...
Obtenidos 700 personajes...
Obtenidos 800 personajes...
Obtenidos 900 personajes...
Obtenidos 1000 personajes...
Obtenidos 1100 personajes...
Obtenidos 1200 personajes...
Obtenidos 1300 personajes...
Obtenidos 1400 personajes...
Obtenidos 1500 personajes...
Obtenidos 1564 personajes...


,id,name,description,modified,comics_available,series_available,stories_available,events_available,event_available,thumbnail
0,1011334,3-D Man,,2014-04-29T14:18:17-0400,12,3,21,21,1,http://i.annihil.us/u/prod/marvel/i/mg/c/e0/53...
1,1017100,A-Bomb (HAS),Rick Jones has been Hulk's best bud since day ...,2013-09-18T15:54:04-0400,4,2,7,7,0,http://i.annihil.us/u/prod/marvel/i/mg/3/20/52...
2,1009144,A.I.M.,AIM is a terrorist organization bent on destro...,2013-10-17T14:41:30-0400,53,36,57,57,0,http://i.annihil.us/u/prod/marvel/i/mg/6/20/52...
3,1010699,Aaron Stack,,1969-12-31T19:00:00-0500,14,3,27,27,0,http://i.annihil.us/u/prod/marvel/i/mg/b/40/im...
4,1009146,Abomination (Emil Blonsky),"Formerly known as Emil Blonsky, a spy of Sovie...",2012-03-20T12:32:12-0400,58,28,68,68,1,http://i.annihil.us/u/prod/marvel/i/mg/9/50/4c...


In [16]:
characters_df.to_csv('Marvel_characters.csv', index = False)

In [17]:
characters_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1564 entries, 0 to 1563
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 1564 non-null   int64 
 1   name               1564 non-null   object
 2   description        1564 non-null   object
 3   modified           1564 non-null   object
 4   comics_available   1564 non-null   int64 
 5   series_available   1564 non-null   int64 
 6   stories_available  1564 non-null   int64 
 7   events_available   1564 non-null   int64 
 8   event_available    1564 non-null   int64 
 9   thumbnail          1564 non-null   object
dtypes: int64(6), object(4)
memory usage: 122.3+ KB


# WEB SCRAPING SUPERHEROES AND VILLAINS

In [24]:
import requests
from bs4 import BeautifulSoup

# URL de la página del héroe
hero_url = "https://www.superherodb.com/a-fourth-raikage/10-18757/"

# Enviar solicitud HTTP a la página del héroe
response = requests.get(hero_url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Analizar el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer el nombre del héroe
    name_tag = soup.find('h1')
    hero_name = name_tag.text.strip() if name_tag else 'Nombre no encontrado'
    
    # Mostrar el nombre para verificar que lo estamos extrayendo correctamente
    print(f"Nombre del Héroe: {hero_name}")
    
    # Extraer las estadísticas del héroe
    stats = {}
    
    # Buscar la sección de estadísticas con la clase 'grid'
    stats_section = soup.find('div', class_='grid')
    if stats_section:
        # Encontrar todos los elementos con la clase 'grid-stat'
        stat_items = stats_section.find_all('div', class_='grid-stat')
        
        for item in stat_items:
            # Depurar cada 'grid-stat' encontrado
            attribute = item.find('div', class_='attribute')
            value = item.find('div', class_='value')
            
            if attribute and value:
                stat_name = attribute.text.strip()
                stat_value = value.text.strip()
                stats[stat_name] = stat_value

    # Mostrar las estadísticas para verificar la extracción
    print("Estadísticas:")
    for stat_name, stat_value in stats.items():
        print(f"{stat_name}: {stat_value}")
else:
    print("Error al acceder a la página del héroe")


Nombre del Héroe: A (Fourth Raikage)
Estadísticas:
